In [ ]:
#import packages
import os
import glob
from nipype.interfaces.io import DataGrabber, DataSink, SelectFiles # Data i/o
from nipype.interfaces.utility import IdentityInterface     # utility
from nipype.pipeline.engine import Node, Workflow        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim

from nipype.interfaces.fsl import SliceTimer, FSLCommand, Reorient2Std
from nipype.interfaces.freesurfer import Resample
from nipype.interfaces.ants import N4BiasFieldCorrection, Registration

import nipype.algorithms.modelgen as model   # model generation
import nipype.algorithms.rapidart as ra      # artifact detection

#set output file ot type for FSL to compressed NIFTI
FSLCommand.set_default_output_type('NIFTI')

# Set study variables
studyhome = '/Volumes/iang/active/BABIES/BABIES_rest/pypelineDev'
raw_data = studyhome + '/raw'
output_dir = studyhome + '/processed/preproc'
template_brain = '/Volumes/iang/active/BABIES/infant_templates/nihpd_05-08mo_brain/nihpd_0508_t2w_brain_2mm.nii'

subjects_list = ['035-BABIES-T1']#os.listdir(raw_data)
vols_to_trim = 4
interleave = False
TR = 2 # in seconds
slice_dir = 3 # 1=x, 2=y, 3=z
proc_cores = 4 # number of cores of processing for the workflows
resampled_voxel_size = (2,2,2)

In [ ]:
## File handling Nodes

# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name="infosource")
infosource.iterables = ('subject_id', subjects_list)


# Data grabber- select fMRI and sMRI
templates = {'struct': raw_data + '/{subject_id}/anat/skullstripped_anat.nii',
            'func': raw_data + '/{subject_id}/rest/rest_raw.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

In [ ]:
## Nodes for preprocessing

# Reorient to standard space
reorientfunc = Node(Reorient2Std(), name='reorientfunc')
reorientstruct = Node(Reorient2Std(), name='reorientstruct')

# Trim first 4 volumes
trimvols = Node(Trim(begin_index=vols_to_trim), name='trimvols')

#slice timing correction based on interleaved acquisition
slicetime_correct = Node(SliceTimer(interleaved=interleave, slice_direction=slice_dir,
                                   time_repetition=TR),
                            name='slicetime_correct')

# Motion correction- MEL

# Bias Correction- input image is 'input_image' output is 'output_image'
n4_correct = Node(N4BiasFieldCorrection(save_bias=True), 
                  name='n4_correct')

# Reslice- using MRI_convert 
reslice_struct = Node(Resample(voxel_size=resampled_voxel_size), 
                       name='reslice_struct')

# Smoothing- LK

# Registration- using ANTs. 
# The BOLD image is 'moving_image', the anat is the 'fixed_image', the output is 'warped_image'.
coreg = Node(Registration(collapse_output_transforms=True,
                          initial_moving_transform_com=True,
                          num_threads=1,
                          output_inverse_warped_image=True,
                          output_warped_image=True,
                          sigma_units=['vox']*3,
                          transforms=['Rigid', 'Affine', 'SyN'],
                          terminal_output='file',
                          winsorize_lower_quantile=0.005,
                          winsorize_upper_quantile=0.995,
                          convergence_threshold=[1e-08, 1e-08, -0.01],
                          convergence_window_size=[20, 20, 5],
                          metric=['Mattes', 'Mattes', ['Mattes', 'CC']],
                          metric_weight=[1.0, 1.0, [0.5, 0.5]],
                          number_of_iterations=[[10000, 11110, 11110],
                                                [10000, 11110, 11110],
                                                [100, 30, 20]],
                          radius_or_number_of_bins=[32, 32, [32, 4]],
                          sampling_percentage=[0.3, 0.3, [None, None]],
                          sampling_strategy=['Regular',
                                             'Regular',
                                             [None, None]],
                          shrink_factors=[[3, 2, 1],
                                          [3, 2, 1],
                                          [4, 2, 1]],
                          smoothing_sigmas=[[4.0, 2.0, 1.0],
                                            [4.0, 2.0, 1.0],
                                            [1.0, 0.5, 0.0]],
                          transform_parameters=[(0.1,),
                                                (0.1,),
                                                (0.2, 3.0, 0.0)],
                          use_estimate_learning_rate_once=[True]*3,
                          use_histogram_matching=[False, False, False], #Set to false between different modalities
                          write_composite_transform=True)
             name='coreg')

In [ ]:
## Preprocessing Workflow

# workflowname.connect([(node1,node2,[('node1output','node2input')]),
#                    (node2,node3,[('node2output','node3input')])
#                    ])

preprocwf = Workflow(name='preprocwf')
preprocwf.connect([(infosource,selectfiles,[('subject_id','subject_id')]), 
                   (selectfiles,reorientfunc,[('func','in_file')]),
                   (reorientfunc,trimvols,[('out_file','in_file')]),
                   (trimvols,datasink,[('out_file','trimmed')])
                  ])
preprocwf.base_dir = output_dir
preprocwf.write_graph(graph2use='flat')
preprocwf.run('MultiProc', plugin_args={'n_procs': proc_cores})